In [3]:
import numpy as np


## Dataset Details

- Size: 382,219 vertices (actors)
- Volume: 33,115,812 edges (collaborations)
- Unique volume: 30,076,166 edges (collaborations)
- Average degree: 173.28 edges / vertex
- Wedge count: 6,266,209,411 
- Triangle count	346,813,199 


A common measure is the transitivity = 3T/W

In [4]:
import os

def stream(stream_length = 10): 
    path = "../data/actor-collaboration/out.actor-collaboration"
    raw_documents = []
    i = 0
    with open(path, encoding='utf-8', errors='replace') as f:
        while i < stream_length:
            line = f.readline()
            print(line)
            i += 1


% sym positive

1 2 

2 1 

1 3 

1 4 

1 5 

1 6 

1 7 

3 1 

3 4 



### Core algorithm

#### Wedge sampling to estimate the transitivity
A wedge is closed if it participates in a triangle, otherwise it is open. Note that the transitivity (= 3T/W) is exactly the probability that a uniform random wedge is closed. This leads to a simple randomized algorithm for estimating transivity (and triangle count) by sampling a set of independent uniform random wedges and finding the fraction of closed wedges. 

#### But how to sample wedges from a stream of edges? 
Suppose we just sampled a uniform random set of edges. How large does this set need to be to get a wedge? The birthday paradox can be used to deduce that (as long as W ≥ m, which holds for a great majority, if not all, of real networks) O(sqrt(n)) edges suffice


#### Reservoir sampling 
Reservoir sampling means sampling a stream sequentially, with probability 1/i to pick the i:th data point. 

#### Main data structures 

- Array edge res[1...se]: This is the array of reservoir edges and is the subsample of the stream maintained.
- New wedges Nt: This is a list of all wedges involving et formed only by edges in edge res. This may often be empty, if et is not added to the edge res. We do not necessarily maintain this list explicitly, and we discuss implementation details later.
- Variable tot wedges: This is the total number of wedges formed by edges in the current edge res.
- Array wedge res[1...sw]: This is an array of reservoir wedges of size sw.
- Array isClosed[1...sw]: This is a boolean array. We set isClosed[i] to be true if wedge wedge res[i] is detected as closed.

#### Algorithm
![update.png](./update.png)

In [43]:
class StreamingTriangles: 
    def __init__(this, path, se, sw, early_stop = np.inf): 
        ## Initialize edge_res of size se and wedge_res of size sw
        this.se = se
        this.sw = sw
        
        this.edge_res = [(0,0)] * se
        this.wedge_res = [((0,0), (0,0))] * sw # containing the sampled wedges, a tuple of two tuples

        this.tot_wedges = 0 # the number of wedges formed by edges in the current edge res
        
        this.isClosed = np.zeros((1,sw), dtype = bool)
        
        this.K = [0]
        this.T = [0]
        
        this.t = 1 # t denotes the timestep 

        ## Simulate stream
        with open(path, encoding='utf-8', errors='replace') as f:

            f.readline() # skip first line 
            
            while this.t < early_stop: 
                ## read data from line
                line = f.readline()
                edge = (int(line.split(" ")[0]), int(line.split(" ")[1]))
                
                # call update 
                this.update(edge)
                
                # let p be the fraction of entries in isClosed set to true
                p = this.isClosed.sum()/len(this.isClosed)
                
                Kt = 3 * p
                
                Tt = (p * t**2 / se * (se-1) ) * this.tot_wedges
                
                this.t += 1
                
    def update(this, et): 
        new_edges = 0
        new_wedges = 0
        
        ## Check if et creates any triangles
        for i in range(this.sw): 
            if this.isTriangle(wedge_res[i], et): ## if wedge_res[i] closed by et:
                this.isClosed[i] = True
        
        ## Randomly update the sampled edges 
        for i in range(this.se):
            
            #pick a random number x in [0, 1]
            x = np.random.rand()
            
            if (x <= i/this.t): 
                this.edge_res[i] = et
                new_edges += 1
                
        if new_edges:             
            
            # get all wedges formed by the edges in edge_res
            all_wedges = this.getAllWedges()
            
            # update tot_wedges: the number of wedges in this.edge_res
            this.tot_wedges = len(all_wedges)
            
            # determine Nt (wedges involving et) and let new_wedges = |Nt|
            Nt = this.getWedgesWithEdge(all_wedges, et)
            new_wedges = len(Nt)
            
            for i in range(this.sw): 
                #pick a random number x in [0, 1]
                x = np.random.rand()   
                
                if (x <= new_wedges/this.tot_wedges):
                    # pick uniform random w in Nt
                    w = Nt[int(np.random.rand() * len(Nt))]
                    
                    this.wedge_res[i] = w
                    
                    this.isClosed[i] = False
        
        
        
    
        
    def isTriangle(this, wedge, edge): 
        ## determines if a wedge and an edge forms a triange   
        
        if (edge[0] == edge[1]): 
            print("Found stupid edge {}".format(edge))
            return False
        
        e1 = set(wedge[0])
        e2 = set(wedge[1])
        leaves = e1 ^ e2
        
        if ((edge[0] in leaves) and (edge[1] in leaves)):
            return True 
        else: 
            return False
        
    def getWedgesWithEdge(this, wedges, edge):
        wedges_with_edge = set()
        for wedge in wedges: 
            if (edge in wedge): 
                wedges_with_edge.add(wedge)
                
        return wedges_with_edge
                
    
    def getAllWedges(this): 
        ## returns all wedges formed by the edges in edge_res
        return False 

In [31]:
## Hyperparams 
se = 10
sw = 10


path = "../data/actor-collaboration/out.actor-collaboration"

In [18]:


ST = StreamingTriangles(path, se, sw, 100)



(1, 2)
(2, 1)
(1, 3)
(1, 4)
(1, 5)
(1, 6)
(1, 7)
(3, 1)
(3, 4)
(3, 5)


In [44]:
## Tests


ST = StreamingTriangles(path, se, sw, 0)

wedges = ((0, 1), (1, 2))
trueEdge = (2, 0)
falseEdge = (2, 1)
stupidEdge = (1, 1)

print("{}: expected: True".format(ST.isTriangle(wedges, trueEdge)))

print("{}: expected: False".format(ST.isTriangle(wedges, falseEdge)))

print("{}: expected: False".format(ST.isTriangle(wedges, stupidEdge)))






True: expected: True
False: expected: False
Found stupid edge (1, 1)
False: expected: False
